# Traffic Sign Classifier


In [1]:
### import training log file

import csv

lines=[]

with open('/home/carnd/P3_sankar/myData/data/driving_log.csv')as csvfile:
    reader=csv.reader(csvfile)
    for line in reader:
        lines.append(line)
print("file load successful... first line is ",lines[0])


file load successful... first line is  ['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed']


In [2]:
### Separate training data into images and measurements(steering angle, pedal, brake etc..)
### Based on an image obtained via front,left or right camera, then network predicts steer angle
### X_train will be images, Y_train is steer angle

import cv2
import numpy as np

images=[]
measurements=[]

lines.pop(0) ## remove labels

for line in lines:
    for i in range(3):
        source_path=line[i]
        filename=source_path.split('/')[-1]
        current_path='/home/carnd/P3_sankar/myData/data/IMG/'+ filename
        image=cv2.imread(current_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if image == None:
            print("Invalid image:" , current_path)
        else:
            images.append(image)
            measurement = float(line[3])
            if i==1:
                measurements.append(measurement+0.2) ## for a left image, steer right a bit
            elif i==2:
                measurements.append(measurement-0.2) ## for a right image, steer left a bit
            else:
                measurements.append(measurement)   ## for a center image, do nothing             
                

## Test any specific training point as needed    
print("base data set measurement size",len(measurements))
print("base data set # of images",len(images))
print()

## Augmenting images to the data set

# Flip images and measurement

images_aug=[]
measurements_aug=[]

for image,measurement in zip(images,measurements):
    images_aug.append(image)    
    measurements_aug.append(measurement)
    images_aug.append(cv2.flip(image,1))
    measurements_aug.append(measurement*-1.0)

print("Aug data set measurement size",len(measurements_aug))
print("Aug data set # of images",len(images_aug))
print()


/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:20: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


base data set measurement size 24108
base data set # of images 24108

Aug data set measurement size 48216
Aug data set # of images 48216



In [3]:
### Train a simple network based on images and steer angle

x_train=np.array(images_aug) ## converting to numpy arrays for tensorflow
y_train=np.array(measurements_aug)

from keras.models import Sequential
from keras.layers import Flatten,Dense
from keras.layers import Lambda,Cropping2D,Convolution2D

print("Image shape is",x_train[0].shape)

model=Sequential()
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Convolution2D(24,5,5,subsample=(2,2),activation="relu"))
model.add(Convolution2D(36,5,5,subsample=(2,2),activation="relu"))
model.add(Convolution2D(48,5,5,subsample=(2,2),activation="relu"))
model.add(Convolution2D(64,3,3,activation="relu"))
model.add(Convolution2D(64,3,3,activation="relu"))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse',optimizer='adam')
model.fit(x_train,y_train,validation_split=0.2,shuffle=True,nb_epoch=3,verbose=1)

model.save('SankarModel.h5')
print('model saved')





Using TensorFlow backend.


Image shape is (160, 320, 3)
Train on 38572 samples, validate on 9644 samples
Epoch 1/3
38572/38572 [==============================] - 92s - loss: 0.0174 - val_loss: 0.0199
Epoch 2/3
38572/38572 [==============================] - 79s - loss: 0.0149 - val_loss: 0.0206
Epoch 3/3
38572/38572 [==============================] - 79s - loss: 0.0136 - val_loss: 0.0189
model saved
